# Subtask 1 : Fetching tweets of @MIDASIIIITD

In order to fetch tweets through Twitter API, one needs to register an App through their twitter account.
Once the app is created, you will be redirected to the app page.
Open the ‘Keys and Access Tokens’ tab.
Copy ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’.

Importing Essential Libraries

In [12]:
import sys
import json
import requests
from requests_oauthlib import OAuth1

Authentication for access of Twitter API

In [2]:
def get_authenticated():

    CONSUMER_KEY = "6JjGBmFAzrlOaVLbQNCp4h8wZ"
    CONSUMER_SECRET = "QQSiJQOu91jDkO5DzOtnpUNNiQ320O06hdcGXDANqvD79UKWPa"
    ACCESS_KEY = "1494681510-FCNDJLlEXFEx4W0C0nOyaMQNfvWwccrINf7oDGI"
    ACCESS_SECRET = "j8RrVRELkPjf2Au4G3A0PxPff7Lfys77bfVkoeYLPYNCT"

    auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET)

    return auth

Since twitter by default allow only 20 tweets on one request if count parameters is not given any value. Therefore in one request we have to get the total no. of tweets/retweets by an user


In [3]:
def get_count_of_tweet(user_handle, URL, auth):

    URL = URL + "?screen_name={}".format(user_handle)

    try:
        response = requests.get(URL, auth=auth)
    except requests.exceptions.RequestException:
        print('No internet connection')
        sys.exit(1)

    data = response.json()
    count = data[0]["user"]["statuses_count"]

    return count

get_json_data send the GET requests for fetching the tweets. This function accepts userhandle, nooftweets, and base URL and returns the parsed JSON data


In [4]:
def get_json_data(user_handle, URL, no_of_tweets, auth):

    URL = URL + \
        "?screen_name={}&tweet_mode=extended&count={}".format(
            user_handle, no_of_tweets)

    try:
        response = requests.get(URL, auth=auth)
    except requests.exceptions.RequestException:
        print('No internet connection')
        sys.exit(1)

    data = response.json()

    return data

save_data_into_file accepts the JSON data and save it into file named tweet.json

In [5]:
def save_data_into_file(data):

    with open('tweet.json', 'w') as f:
        json.dump(data, f, indent=4)

main calls all the required function and finish the subtask

In [6]:
def main():

    user_handle = "midasIIITD"
    URL = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

    auth = get_authenticated()
    no_of_tweets = get_count_of_tweet(user_handle, URL, auth)
    data = get_json_data(user_handle, URL, no_of_tweets, auth)
    save_data_into_file(data)


if __name__ == '__main__':
    main()

# Subtask 2 : Parsing these JSONline files and displaying every tweet in tabular format

In [7]:
def get_all_media_count(tweet):
    
    count = 0
    
    if 'extended_entities' in tweet:
        try:
            count += len(tweet['extended_entities']['media'])
        except KeyError:
            pass
    
    elif 'media' in tweet['entities']:
        count += len(tweet['entities']['media'])
    
    if 'retweeted_status' in tweet:
        count += get_all_media_count(tweet['retweeted_status'])
    
    return count

BeautifulTable allow to print data in tabular manner this block import neccessary lib and print the tweets in tabular form

In [9]:
from beautifultable import BeautifulTable
table = BeautifulTable()
table.column_headers = ['Text', 'Date and Time', 'No. of favorites', 'No. of Retweets', 'No. of images']

with open('tweet.json', 'r') as f:
    json_data = f.read()

data = json.loads(json_data)

for tweet in data:
        
    count = get_all_media_count(tweet)
    if count == 0:
        count = None

    tweet_data = [tweet['full_text'], tweet['created_at'], tweet['favorite_count'], tweet['retweet_count'],
                  count]
    
    table.append_row(tweet_data)
    
print(table)

+---------------------------------------------------+-------+-----+-------+----+
|                       Text                        | Date  | No. | No. o | No |
|                                                   | and T |  of | f Ret | .  |
|                                                   |  ime  |  fa | weets | of |
|                                                   |       | vor |       |  i |
|                                                   |       | ite |       | ma |
|                                                   |       |  s  |       | ge |
|                                                   |       |     |       | s  |
+---------------------------------------------------+-------+-----+-------+----+
| Many Congratulations to @midasIIITD student, Shag | Mon A |  7  |   2   | 1  |
| un Uppal @shagunuppls, on getting selected for th | pr 08 |     |       |    |
| e summer internship in the BRAIN lab at Singapore |  07:0 |     |       |    |
|  University of Technology 